## 2-1. 데이터 읽기

In [30]:
import numpy as np
import pandas as pd


# u.user 파일을 DataFrame으로 읽기 
u_cols = ['user_id', 'age', 'sex', 'occupation', 'zip_code']
users = pd.read_csv('../Data/u.user', sep='|', names=u_cols, encoding='latin-1')
users = users.set_index('user_id')
users.head()

,age,sex,occupation,zip_code
user_id,,,,
1,24,M,technician,85711
2,53,F,other,94043
3,23,M,writer,32067
4,24,M,technician,43537
5,33,F,other,15213


In [31]:
# u.item 파일을 DataFrame으로 읽기
i_cols = ['movie_id', 'title', 'release date', 'video release date', 'IMDB URL', 
        'unknown', 'Action', 'Adventure', 'Animation', 'Children\'s', 'Comedy', 
        'Crime', 'Documentary', 'Drama', 'Fantasy', 'Film-Noir', 'Horror', 
        'Musical', 'Mystery', 'Romance', 'Sci-Fi', 'Thriller', 'War', 'Western']
movies = pd.read_csv('../Data/u.item', sep='|', 
                names=i_cols, encoding='latin-1') # names: dataframe의 열 이름으로 할당
# 'movie_id'라는 열을 DataFrame의 인덱스로 설정
movies = movies.set_index('movie_id')   # movie_id 열을 인덱스로 설정
movies.head()

,title,release date,video release date,IMDB URL,unknown,Action,Adventure,Animation,Children's,Comedy,...,Fantasy,Film-Noir,Horror,Musical,Mystery,Romance,Sci-Fi,Thriller,War,Western
movie_id,,,,,,,,,,,,,,,,,,,,,
1,Toy Story (1995),01-Jan-1995,NaN,http://us.imdb.com/M/title-exact?Toy%20Story%2...,0,0,0,1,1,1,...,0,0,0,0,0,0,0,0,0,0
2,GoldenEye (1995),01-Jan-1995,NaN,http://us.imdb.com/M/title-exact?GoldenEye%20(...,0,1,1,0,0,0,...,0,0,0,0,0,0,0,1,0,0
3,Four Rooms (1995),01-Jan-1995,NaN,http://us.imdb.com/M/title-exact?Four%20Rooms%...,0,0,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0
4,Get Shorty (1995),01-Jan-1995,NaN,http://us.imdb.com/M/title-exact?Get%20Shorty%...,0,1,0,0,0,1,...,0,0,0,0,0,0,0,0,0,0
5,Copycat (1995),01-Jan-1995,NaN,http://us.imdb.com/M/title-exact?Copycat%20(1995),0,0,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0


In [32]:
# u.data 파일을 DataFrame으로 읽기 (rating 1~5점/timestap 연도/날짜/시간을 숫자로 변환한 값)
r_cols  = ['user_id', 'movie_id', 'rating', 'timestamp']
ratings = pd.read_csv('../Data/u.data', sep='\t', names=r_cols, encoding='latin-1') # tab으로 분리
ratings = ratings.set_index('user_id')
ratings.head()

,movie_id,rating,timestamp
user_id,,,
196,242,3,881250949
186,302,3,891717742
22,377,1,878887116
244,51,2,880606923
166,346,1,886397596


## 2-2. 인기제품(best-seller) 방식

In [33]:
# Best-seller 추천 

def recom_movie1(n_items=5):
    # movie_mean을 내림차순으로 n_items 만큼 movie_sort에 저장
    movie_sort      = movie_mean.sort_values(ascending=False)[:n_items]
    # movies에서 movie_sort에 해당하는 정보를 recom_movie에 저장
    # df.loc[index(, column)], df.loc[[index_labels], [column_labels]]
    recom_movies    = movies.loc[movie_sort.index]
    # recom_movies에서 영화제목만 골라내어 저장
    recommendations = recom_movies['title']
    return recommendations

# 각 movie_id 별 rating의 평균(mean)을 계산하여 저장 (Series 형식)
movie_mean = ratings.groupby(['movie_id'])['rating'].mean()
print(recom_movie1(5))

# recom_movie1과 같은 역할. 코드를 한 줄로 만든 ver.
def recom_movie2(n_items):
    return movies.loc[movie_mean.sort_values(ascending=False)[:n_items].index]['title']

print(recom_movie2(5))

movie_id
814                         Great Day in Harlem, A (1994)
1599                        Someone Else's America (1995)
1201           Marlene Dietrich: Shadow and Light (1996) 
1122                       They Made Me a Criminal (1939)
1653    Entertaining Angels: The Dorothy Day Story (1996)
Name: title, dtype: object
movie_id
814                         Great Day in Harlem, A (1994)
1599                        Someone Else's America (1995)
1201           Marlene Dietrich: Shadow and Light (1996) 
1122                       They Made Me a Criminal (1939)
1653    Entertaining Angels: The Dorothy Day Story (1996)
Name: title, dtype: object


## 2-3. 추천 시스템의 정확도 측정

In [34]:
# 정확도 계산 
def RMSE(y_true, y_pred):   
    return np.sqrt(np.mean((np.array(y_true) - np.array(y_pred))**2))

rmse = []
for user in set(ratings.index): # ratings에 있는 user_id를 다 추출해서 user마다 루프
    # 각 user가 평가한 영화의 평점을 y_true에 저장
    y_true = ratings.loc[user]['rating']    
    # 해당 user가 평가한 영화의 전체 평점 평균을 y_pred에 저장
    y_pred = movie_mean[ratings.loc[user]['movie_id']]  
    # 해당 user의 rmse 계산 (예측값(전체유저의 평점평균), 실제값 (해당유저의 평점))
    accuracy = RMSE(y_true, y_pred)
    # 현재 사용자의 rmse(정확도)를 리스트에 추가
    rmse.append(accuracy)

# 전체 사용자의 rmse의 평균
print(np.mean(rmse))

0.996007224010567


## 2-4. 사용자 집단별 추천

### 데이터 읽어오고 train, test set 분리

In [35]:
import numpy as np
import pandas as pd

# 데이터 읽어 오기 
u_cols = ['user_id', 'age', 'sex', 'occupation', 'zip_code']
users  = pd.read_csv('../Data/u.user', sep='|', names=u_cols, encoding='latin-1')
i_cols = ['movie_id', 'title', 'release date', 'video release date', 'IMDB URL', 'unknown', 
        'Action', 'Adventure', 'Animation', 'Children\'s', 'Comedy', 'Crime', 'Documentary', 
        'Drama', 'Fantasy', 'Film-Noir', 'Horror', 'Musical', 'Mystery', 'Romance', 'Sci-Fi', 
        'Thriller', 'War', 'Western']
movies  = pd.read_csv('../Data/u.item', sep='|', names=i_cols, encoding='latin-1')
r_cols  = ['user_id', 'movie_id', 'rating', 'timestamp']
ratings = pd.read_csv('../Data/u.data', sep='\t', names=r_cols, encoding='latin-1')

# timestamp 제거 
ratings = ratings.drop('timestamp', axis=1)     # axis=1:column을 삭제
# movie ID와 title 빼고 다른 데이터 제거
movies  = movies[['movie_id', 'title']]

In [36]:
# train, test set 분리
from sklearn.model_selection import train_test_split
x = ratings.copy()
y = ratings['user_id']  # train_set과 test_set 분리 기준을 user_id로 하기 위해.
# 75% train_set, 25% test_set 분리 / user_id 비율 맞춰서 train과 test 분리
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.25, stratify=y)
print(x_train)

       user_id  movie_id  rating
84155      843       181       3
34747      453       318       4
38509      450       302       5
59571      850       566       5
63084      717       975       2
...        ...       ...     ...
20825      401        71       2
91483      929       135       5
65037      850       228       5
35625       82       866       3
49883      668        97       2

[75000 rows x 3 columns]


/opt/homebrew/lib/python3.11/site-packages/sklearn/utils/validation.py:605: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if is_sparse(pd_dtype):
/opt/homebrew/lib/python3.11/site-packages/sklearn/utils/validation.py:614: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if is_sparse(pd_dtype) or not is_extension_array_dtype(pd_dtype):


### 예측모델별 정확도 RMSE 계산하는 함수

In [37]:
# 정확도(RMSE)를 계산하는 함수
def RMSE(y_true, y_pred):
    return np.sqrt(np.mean((np.array(y_true) - np.array(y_pred))**2))

# 모델별 RMSE를 계산하는 함수 
def score(model):
    # test할 사용자-영화 짝 pair 생성
    id_pairs = zip(x_test['user_id'], x_test['movie_id'])
    # 예측 모델에 의한 사용자-영화 짝에 대한 예측값을 계산(by x_train)하여 y_pred에 저장
    y_pred = np.array([model(user, movie) for (user, movie) in id_pairs])
    # 실제 평점값 리스트(by x_test)를 y_true에 저장
    y_true = np.array(x_test['rating'])
    return RMSE(y_true, y_pred)

# train 데이터로 Full matrix 구하기 
rating_matrix = x_train.pivot(index='user_id', columns='movie_id', values='rating')


### 예측모델 1: 전체 평균으로 예측

In [38]:
# 전체 평균으로 예측치를 계산하는 기본 모델
def best_seller(user_id, movie_id):
    try:
        # x_train에서 해당 movie_id의 rating 전체 평균값을 rating에 저장
        rating = train_mean[movie_id]
    except: # test에 있는 movie_id가 train에 없을 때
        rating = 3.0
    return rating

train_mean = x_train.groupby(['movie_id'])['rating'].mean()
score(best_seller)

1.0231339443139937

2-3에서 전체 데이터를 사용하여 예측한 경우보다 나빠진(RMSE 값이 커진) 이유는 train/test set을 분리하여 자신의 데이터로 자신을 예측하지 않도록 했기 때문!

### 예측모델 2: gender별 평균으로 예측

In [39]:
# Full matrix를 사용자 데이터와 merge (공통key인 user_id 기준으로 merge)
merged_ratings = pd.merge(x_train, users)   # user정보와 rating train set이 합쳐짐
users = users.set_index('user_id')

In [40]:
# gender별 평점평균 계산
# sex와 movie_id를 기준으로 평점(rating)의 평균을 구해서 g_means에 저장
g_mean = merged_ratings[['movie_id', 'sex', 'rating']].groupby(['movie_id', 'sex'])['rating'].mean()

# gender별 평균을 예측치로 돌려주는 모델
def cf_gender(user_id, movie_id):
    if movie_id in rating_matrix:   # test셋에 있는 movie_id가 traing셋(rating_matrix)에 있다면
        gender = users.loc[user_id]['sex']  # test 당하는 user의 성별 반환
        if gender in g_mean[movie_id]:      # 해당 gender가 해당 movie_id를 평가한 데이터가 train셋에 있다면
            gender_rating = g_mean[movie_id][gender]
        else:                               # 해당 gender가 해당 movie_id를 평가한 데이터가 train셋에 없다면
            gender_rating = 3.0
    else:                           # test셋에 있는 movie_id가 traing셋(rating_matrix)에 없다면
        gender_rating = 3.0
    return gender_rating

score(cf_gender)

1.033517203648101

In [41]:
# gender별 평점평균 계산
# sex와 movie_id를 기준으로 평점(rating)의 평균을 구해서 g_means에 저장
g_mean = merged_ratings[['movie_id', 'sex', 'rating']].groupby(['movie_id', 'sex'])['rating'].mean()

# gender별 평균을 예측치로 돌려주는 모델
def cf_gender(user_id, movie_id):
    if movie_id in rating_matrix:   # test셋에 있는 movie_id가 traing셋(rating_matrix)에 있다면
        gender = users.loc[user_id]['sex']  # test 당하는 user의 성별 반환
        if gender in g_mean[movie_id]:      # 해당 gender가 해당 movie_id를 평가한 데이터가 train셋에 있다면
            gender_rating = g_mean[movie_id][gender]
        else:                               # 해당 gender가 해당 movie_id를 평가한 데이터가 train셋에 없다면
            gender_rating = 3.0
    else:                           # test셋에 있는 movie_id가 traing셋(rating_matrix)에 없다면
        gender_rating = 3.0
    return gender_rating

score(cf_gender)

1.033517203648101

### 예측모델 3: gender와 occupation을 동시에 고려한 집단별 평균으로 예측 (연습문제 2-2)

In [42]:
# (gender, occupation) 별 평점평균 계산
# sex와 movie_id를 기준으로 평점(rating)의 평균을 구해서 g_means에 저장
g_o_mean = merged_ratings[['movie_id', 'sex', 'occupation', 'rating']].groupby(['movie_id', 'sex', 'occupation'])['rating'].mean()
print(g_o_mean)

movie_id  sex  occupation   
1         F    administrator    4.0
               artist           4.4
               educator         3.4
               engineer         4.0
               entertainment    4.5
                               ... 
1678      M    student          1.0
1679      M    student          3.0
1680      M    student          2.0
1681      M    writer           3.0
1682      M    engineer         3.0
Name: rating, Length: 22505, dtype: float64


In [43]:
# gender, occupation 별 평균을 예측치로 돌려주는 모델
def cf_gender_occupation(user_id, movie_id):
    if movie_id in rating_matrix:   # test셋에 있는 movie_id가 traing셋(rating_matrix)에 있다면
        gender     = users.loc[user_id]['sex']          # test 당하는 user의 성별 반환
        occupation = users.loc[user_id]['occupation']   # test 당하는 user의 직업 반환
        if gender in g_o_mean[movie_id]:                # 해당 gender가 해당 movie_id를 평가한 데이터가 train셋에 있다면
                if occupation in g_o_mean[movie_id][gender]:    # 해당 gender의 해당 occupation이 해당 movie_id를 평가한 데이터가 train셋에 있다면
                    gender_rating = g_o_mean[movie_id][gender][occupation]
                else:
                    gender_rating = 3.0
        else:                               
            gender_rating = 3.0
    else:                           
        gender_rating = 3.0
    return gender_rating

score(cf_gender_occupation)

1.1356397530010616

사용자 집단(성별, 직업과 같은 인구통계적 변수 기준)별 추천은 정확하지 않다!!!  
3장에서 취향을 기준으로 나누어 보자!

## 2-5. 내용 기반 필터링 추천

In [5]:
# Created or modified on May 2022
# Author: 임일
# CB기반 추천

import pandas as pd

# Data 읽기 (내용기반 -> 줄거리 필요 -> movies_metadata.csv 이용)
movies = pd.read_csv('../Data/movies_metadata.csv', encoding='latin-1', low_memory=False)
movies = movies[['id', 'title', 'overview']]
print(movies.head(10))
len(movies) # 영화개수

      id                        title  \
0    862                    Toy Story   
1   8844                      Jumanji   
2  15602             Grumpier Old Men   
3  31357            Waiting to Exhale   
4  11862  Father of the Bride Part II   
5    949                         Heat   
6  45325                 Tom and Huck   
7   9091                 Sudden Death   
8    710                    GoldenEye   
9   9087       The American President   

                                            overview  
0  Led by Woody, Andy's toys live happily in his ...  
1  When siblings Judy and Peter discover an encha...  
2  A family wedding reignites the ancient feud be...  
3  Cheated on, mistreated and stepped on, the wom...  
4  Just when George Banks has recovered from his ...  
5  Obsessive master thief, Neil McCauley leads a ...  
6  A mischievous young boy, Tom Sawyer, witnesses...  
7  International action superstar Jean Claude Van...  
8  James Bond must unmask the mysterious head of ... 

45442

In [9]:
########## 1. 데이터 전처리
movies = movies.drop_duplicates()   # 중복 데이터 삭제
movies = movies.dropna()            # 비어있는 데이터 삭제
movies['overview'] = movies['overview'].fillna('')  # 줄거리가 비어있는 것은 빈칸으로 메움
print(len(movies))


########## 2. 각 문서(overview)에 등장하는 모든 단어의 가중치 계산
from sklearn.feature_extraction.text import TfidfVectorizer
# 텍스트의 불용어를 english로 지정
tfidf = TfidfVectorizer(stop_words='english')
# 줄거리(overview) 텍스트에 대해서 tf-idf를 계산
# tf: 어떤 단어가 해당 문서에 얼마나 자주 등장하는가 / idf: 다른 문서에 비해 상대적으로 얼마나 자주 등장하는가
tfidf_matrix = tfidf.fit_transform(movies['overview'])

######### 3. 단어의 가중치가 문서 간(영화 줄거리)에 얼마나 유사한지 유사도 계산
from sklearn.metrics.pairwise import cosine_similarity
# 영화 간의 유사도 계산
cosine_sim = cosine_similarity(tfidf_matrix, tfidf_matrix)
# 유사도 다루기 편한 DataFrame으로 변환
cosine_sim = pd.DataFrame(cosine_sim, index=movies.index, columns=movies.index)
print(cosine_sim)


44300
          0         1         2         3         4         5         6      \
0      1.000000  0.014981  0.000000  0.000000  0.000000  0.000000  0.000000   
1      0.014981  1.000000  0.046968  0.000000  0.000000  0.050222  0.000000   
2      0.000000  0.046968  1.000000  0.000000  0.025070  0.000000  0.006414   
3      0.000000  0.000000  0.000000  1.000000  0.000000  0.007214  0.008982   
4      0.000000  0.000000  0.025070  0.000000  1.000000  0.000000  0.000000   
...         ...       ...       ...       ...       ...       ...       ...   
45437  0.000000  0.000000  0.000000  0.026478  0.000000  0.025460  0.000000   
45438  0.000000  0.066866  0.000000  0.000000  0.000000  0.000000  0.000000   
45439  0.000000  0.000000  0.000000  0.000000  0.000000  0.000000  0.000000   
45440  0.005955  0.022018  0.014077  0.009531  0.007014  0.000000  0.005263   
45441  0.000000  0.009356  0.000000  0.016436  0.000000  0.000000  0.000000   

          7         8         9      ...  454

In [10]:
# index-title을 뒤집는다
indices = pd.Series(movies.index, index=movies['title'])

######### 4. 영화제목을 받아서 추천 영화를 돌려주는 함수
def content_recommender(title, n_of_recomm):
    # title에서 영화 index 받아오기
    idx = indices[title]
    # 주어진 영화와 다른 영화의 similarity를 가져온다
    sim_scores = cosine_sim[idx]
    # similarity 기준으로 내림차순 정렬하고 n_of_recomm만큼 가져오기 (자기자신은 빼기)
    sim_scores = sim_scores.sort_values(ascending=False)[1:n_of_recomm+1]
    # 영화 title 반환
    return movies.loc[sim_scores.index]['title']

# 추천받기
print(content_recommender('The Lion King', 5))
print(content_recommender('The Dark Knight Rises', 10))

34664    How the Lion Cub and the Turtle Sang a Song
9339                               The Lion King 1Â½
9101                  The Lion King 2: Simba's Pride
42806                                           Prey
25637                                 Fearless Fagan
Name: title, dtype: object
12468                                      The Dark Knight
149                                         Batman Forever
1321                                        Batman Returns
15497                           Batman: Under the Red Hood
584                                                 Batman
21179    Batman Unmasked: The Psychology of the Dark Kn...
9216                    Batman Beyond: Return of the Joker
18021                                     Batman: Year One
19778              Batman: The Dark Knight Returns, Part 1
3085                          Batman: Mask of the Phantasm
Name: title, dtype: object
